# Ben E Keith COVID Analysis

Goal: 
1. Process BEK data (source = data.attribytes.com)
2. Analyze data using COVID segmentation
3. Compare sell-out (Ben E Keith) to sell-in (McCain) data

### 1. Load libraries, initiate folder/file paths
Run cell below

In [1]:
import pandas as pd
import datetime
from datetime import datetime as dt
import numpy as np
#import teradatasql
import pyodbc

from distributor_transformation import transform_bek
from sellout_model import process_list, analyze, add_time
#from sellout_teradata import teradata_sales
from sellout_import import import_bek, all_df
from sellout_azure import azure_sellin

# Import portal raw file

In [2]:
def setup_connection():
    server = 'mf-enterprise-dev-sql.46ac3df1733c.database.windows.net'
    database = 'PWRAPPDB'
    driver = '{ODBC Driver 17 for SQL Server}'

    # Establish the database connection using AAD Integrated Authentication
    conn_str = (
        f'DRIVER={driver};'
        f'SERVER=tcp:{server};'
        f'DATABASE={database};'
        'Authentication=ActiveDirectoryIntegrated'
    )

    cnxn = pyodbc.connect(conn_str)
    cursor = cnxn.cursor()

    return cnxn, cursor


In [3]:
query_str = '''
SELECT
[Business Unit],
[SIC Code],
[SIC Sub],
[Customer City],
[Customer State],
[McCain SKU ID],
[Week Beginning Date],
SUM([LBS]) as LBS
FROM [PWRAPPDB].[na_dist].[US_BEK_Sellout]
GROUP BY
[Business Unit],
[SIC Code],
[SIC Sub],
[Customer City],
[Customer State],
[McCain SKU ID],
[Week Beginning Date]
'''

cnxn, cursor = setup_connection() 

df = pd.read_sql_query(query_str, cnxn)

# Close the connection
cursor.close()
cnxn.close()

### 7. Apply Dictionary to Raw Data / Check For Missing Segmentation
Run cell below

In [5]:
df_bek = transform_bek(df, 'BEK - US.xlsx')

Shape before adding dictionary: (2795538, 11)
Shape after adding segmentation: (2795538, 14)
Shape after adding product segmentation: (2795538, 17)
Shape before adding time: (2795538, 18)
Shape after adding time: (2795538, 19)
Shape after adding dictionary: (2795538, 20)
Nothing missing for COVID Segmentation - L1
Nothing missing for products


In [6]:
df_bek['Week Starting'].max()

Timestamp('2024-04-14 00:00:00')

### 8. Combine New and Base Datasets
Run cell below

### 9. Execute Analysis
Run cell below

In [9]:
_list = []

#Output 1: COVID L1 - List 0
_list.append(['City','State Name','COVID Segmentation - L1','COVID Segmentation - L2','Restaurant Service Type','Consolidated Category'])

#Output 2: COVID L1 - List 1
_list.append(['State Name','COVID Segmentation - L1','COVID Segmentation - L2','Restaurant Service Type','SKU ID','Consolidated Category','L1 Product Hierarchy','L2 Product Hierarchy'])

#Create dataframes
print(f'Processing Segments', flush = True)
output1 = process_list(df_bek, _list[0], 'Ben E Keith')

#print(f'Processing Sell in vs Sell out', flush = True)
#output2 = teradata_sales(_base, '6500002764', 'Ben E Keith')

Processing Segments


In [11]:
print(f'Processing Sell in vs Sell out', flush = True)
output2 = azure_sellin(df_bek, 'US01', "','".join(['6500002764']), 'Ben E Keith')

print('All done')

Processing Sell in vs Sell out
Query ran for 6500002764 under sales org US01 for sales on or before 2024-04-21
All done, took 59.0 seconds...
All done


In [12]:
output1.to_csv('files/sellout_region_bek.zip', compression='zip', index=False)
output2.to_csv('files/sellout_sellin_bek.zip', compression='zip', index=False)

# Import into SQL mi server

In [3]:
import pandas as pd

In [2]:
def convert_to_dataframe(file_path):

    if file_path.endswith('.csv'):
            df = pd.read_csv(file_name, thousands = ',', encoding="utf-8", low_memory = False, header = 0,na_values = " ")
            df = df[df['Branch'] != 'Total']
    else:
        df = pd.DataFrame()

        _import = pd.read_excel(file_path, sheet_name=None)
        
        for f in _import:
            if f == 'Sheet1':
                add = pd.DataFrame.from_dict(_import[f])
                
                col = add.columns.to_list()
            else:
                add = _import[f].T.reset_index().T
                add.columns = col
        
            df = pd.concat([df, add])
        
        df = df.rename(columns={
            'Unnamed: 21':'LBS',
            'Unnamed: 22':'LBS'
            })

    return df

In [3]:
file_path = r'C:\Users\newatter\OneDrive - McCain Foods Limited\Weekly Update Files\2024-04-12\BEK Update.xls'
bek = convert_to_dataframe(file_path)

In [4]:
bek.columns

Index(['Branch', 'Business Unit', 'SIC Code', 'SIC Sub', 'Customer Nbr.',
       'Customer Name', 'Customer Address1', 'Customer Address2',
       'Customer City', 'Customer State', 'Customer Zip', 'Family', 'Brand',
       'Group', 'Manufacture Prod.Nbr.', 'Prod Nbr', 'Product', 'Prod Nbr.1',
       'Pack / Size', 'Product Ext.Description', 'Week of', 'Month', 'LBS'],
      dtype='object')

In [19]:
columns_to_keep = ['Branch', 'Business Unit', 'SIC Code', 'SIC Sub', 'Customer Nbr.',
       'Customer Name', 'Customer Address1', 'Customer Address2',
       'Customer City', 'Customer State', 'Customer Zip', 'Manufacture Prod.Nbr.', 'Prod Nbr', 'Product', 'Week of', 'LBS']

rename_columns = {
    'Customer Nbr.':'Customer Number',
    'Manufacture Prod.Nbr.':'McCain SKU ID',
    'Prod Nbr':'Product Number',
    'Product':'Product Description',
    'Week of':'Week Beginning Date'
}

nan_rows = bek[bek['Week of'].isna()].index

bek_clean = bek[columns_to_keep].rename(columns=rename_columns).drop(nan_rows)

columns_to_clean = ['Customer Number','Customer Zip','Product Number']

for col in columns_to_clean:
    bek_clean[col] = bek_clean[col].astype(int).astype(str)

bek_clean['LBS'] = bek_clean['LBS'].astype(float)

bek_clean['Week Beginning Date'] = pd.to_datetime(bek_clean['Week Beginning Date'])

bek_clean.head()

,Branch,Business Unit,SIC Code,SIC Sub,Customer Number,Customer Name,Customer Address1,Customer Address2,Customer City,Customer State,Customer Zip,McCain SKU ID,Product Number,Product Description,Week Beginning Date,LBS
1,NEW MEXICO,NATIONAL,RESTAURANTS,HAMBURGER,874113,MOOYAH BURGER 252 EL PASO,8889 GATEWAY BLVD. STE 45,NaN,EL PASO,TX,79925,MCF03731,363445,Potato Sweet Fries 5/16 In,2023-12-31,210.0
2,NEW MEXICO,NATIONAL,RESTAURANTS,HAMBURGER,874113,MOOYAH BURGER 252 EL PASO,8889 GATEWAY BLVD. STE 45,NaN,EL PASO,TX,79925,MCF03731,363445,Potato Sweet Fries 5/16 In,2024-01-07,210.0
3,NEW MEXICO,NATIONAL,RESTAURANTS,HAMBURGER,874113,MOOYAH BURGER 252 EL PASO,8889 GATEWAY BLVD. STE 45,NaN,EL PASO,TX,79925,MCF03731,363445,Potato Sweet Fries 5/16 In,2024-01-14,180.0
4,NEW MEXICO,NATIONAL,RESTAURANTS,HAMBURGER,874113,MOOYAH BURGER 252 EL PASO,8889 GATEWAY BLVD. STE 45,NaN,EL PASO,TX,79925,MCF03731,363445,Potato Sweet Fries 5/16 In,2024-01-21,150.0
5,NEW MEXICO,NATIONAL,RESTAURANTS,HAMBURGER,874113,MOOYAH BURGER 252 EL PASO,8889 GATEWAY BLVD. STE 45,NaN,EL PASO,TX,79925,MCF03731,363445,Potato Sweet Fries 5/16 In,2024-02-04,210.0


In [22]:
import os

# The path to the directory containing the folders
directory_path = r'C:\Users\newatter\OneDrive - McCain Foods Limited\Weekly Update Files'

# Get a list of all entries in the directory
entries = os.listdir(directory_path)

# Filter the list to include only directories, not files
directories = [d for d in entries if os.path.isdir(os.path.join(directory_path, d))]

# Sort the directories by their names, assuming they are in 'yyyy-mm-dd' format
sorted_directories = sorted(directories, key=lambda x: x)

# Loop through the sorted directories
for folder in sorted_directories:
    print(f"Processing folder: {folder}")

Processing folder: 2020-11-13
Processing folder: 2020-11-20
Processing folder: 2020-12-04
Processing folder: 2020-12-11
Processing folder: 2020-12-18
Processing folder: 2021-01-08
Processing folder: 2021-01-15
Processing folder: 2021-01-22
Processing folder: 2021-01-29
Processing folder: 2021-02-05
Processing folder: 2021-02-12
Processing folder: 2021-02-19
Processing folder: 2021-02-26
Processing folder: 2021-03-05
Processing folder: 2021-03-12
Processing folder: 2021-03-19
Processing folder: 2021-03-26
Processing folder: 2021-04-09
Processing folder: 2021-04-16
Processing folder: 2021-04-23
Processing folder: 2021-04-30
Processing folder: 2021-05-05
Processing folder: 2021-05-14
Processing folder: 2021-05-21
Processing folder: 2021-05-28
Processing folder: 2021-06-04
Processing folder: 2021-06-11
Processing folder: 2021-06-18
Processing folder: 2021-06-25
Processing folder: 2021-07-02
Processing folder: 2021-07-09
Processing folder: 2021-07-16
Processing folder: 2021-07-23
Processing

In [1]:
import pyodbc
import sqlalchemy
from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy.engine import URL
from datetime import datetime
import pandas as pd

class bek_sellout:
    def __init__(self, file_path) -> None:
        self.filepath = file_path
        self.data = self.process_file()
        self.data_import = self.filter_rows()
        #self.delete_rows()
        self.insert_rows()

        pass
    
    def transform_data(df):
        columns_to_keep = ['Branch', 'Business Unit', 'SIC Code', 'SIC Sub', 
                           'Customer Nbr.','Customer Name', 'Customer Address1', 'Customer Address2',
                           'Customer City', 'Customer State', 'Customer Zip', 'Manufacture Prod.Nbr.', 
                           'Prod Nbr', 'Product', 'Week of', 'LBS']

        rename_columns = {
            'Customer Nbr.':'Customer Number',
            'Manufacture Prod.Nbr.':'McCain SKU ID',
            'Prod Nbr':'Product Number',
            'Product':'Product Description',
            'Week of':'Week Beginning Date'
        }

        nan_rows = df[df['Week of'].isna()].index

        df_clean = df[columns_to_keep].rename(columns=rename_columns).drop(nan_rows)

        columns_to_clean = ['Customer Number','Customer Zip','Product Number']

        for col in columns_to_clean:
            df_clean[col] = df_clean[col].astype(str)
            df_clean[col] = df_clean[col].str.replace(r'\.0$', '', regex=True)
            
        df_clean['LBS'] = df_clean['LBS'].astype(float)

        df_clean['Week Beginning Date'] = pd.to_datetime(df_clean['Week Beginning Date'])

        return df_clean
    
    def process_file(self):
        if file_path.endswith('.csv'):
            df = pd.read_csv(file_name, thousands = ',', encoding="utf-8", low_memory = False, header = 0,na_values = " ")
            df = df[df['Branch'] != 'Total']
        else:
            df = pd.DataFrame()

            _import = pd.read_excel(file_path, sheet_name=None)
            
            for f in _import:
                if f == 'Sheet1':
                    add = pd.DataFrame.from_dict(_import[f])
                    
                    col = add.columns.to_list()
                else:
                    add = _import[f].T.reset_index().T
                    add.columns = col
            
                df = pd.concat([df, add])

            df = df.rename(columns={
                'Unnamed: 16':'LBS',
                'Unnamed: 17':'LBS',
                'Unnamed: 18':'LBS',
                'Unnamed: 19':'LBS',
                'Unnamed: 20':'LBS',
                'Unnamed: 21':'LBS',
                'Unnamed: 22':'LBS',
                'Unnamed: 23':'LBS'
                })

        return bek_sellout.transform_data(df)

    def setup_connection():
        server = 'mf-enterprise-dev-sql.46ac3df1733c.database.windows.net'
        database = 'PWRAPPDB'
        driver = '{ODBC Driver 17 for SQL Server}'

        # Establish the database connection using AAD Integrated Authentication
        conn_str = (
            f'DRIVER={driver};'
            f'SERVER=tcp:{server};'
            f'DATABASE={database};'
            'Authentication=ActiveDirectoryIntegrated'
        )

        cnxn = pyodbc.connect(conn_str)
        cursor = cnxn.cursor()

        return cnxn, cursor

    def delete_rows(self):
        # Get the unique dates as strings
        unique_dates = bek.data['Week Beginning Date'].dt.strftime('%Y-%m-%d').unique()

        # Convert the array of unique dates to a list of strings
        # Changed due to error received
        #date_strings = [f"'{date}'" for date in unique_dates]

        # Convert the results into a list of datetime objects
        # Adjust the format string as per your date format in the database
        dates_in_db = [datetime.strptime(result[0], '%Y-%m-%d') if result[0] is not None else None for result in results]

        # If you want to create a string that can be directly used in a SQL query
        dates_for_sql = ', '.join(date_strings)

        sql_delete = """
        DELETE FROM [na_dist].[US_BEK_Sellout] 
        WHERE [Week Beginning Date] IN (""" + dates_for_sql + """);"""

        cnxn, cursor = bek_sellout.setup_connection()

        cursor.execute(sql_delete)

        # Commit the transactions
        cnxn.commit()

        # Close the connection
        cursor.close()
        cnxn.close()


    def filter_rows(self):
        sql_select = """
        SELECT [Week Beginning Date] 
        FROM [na_dist].[US_BEK_Sellout]
        GROUP BY [Week Beginning Date] """

        cnxn, cursor = bek_sellout.setup_connection()

        cursor.execute(sql_select)

        # Fetch the results
        results = cursor.fetchall()

        # Convert the results into a list of dates
        dates_in_db = [result[0] if result[0] is not None else None for result in results]
        dates_in_db = pd.to_datetime([date for date in dates_in_db if date is not None])

        # Commit the transactions
        cnxn.commit()

        # Close the connection
        cursor.close()
        cnxn.close()

        return self.data[~self.data['Week Beginning Date'].isin(dates_in_db)]

    def setup_miengine():
        server = 'mf-enterprise-dev-sql.46ac3df1733c.database.windows.net'
        database = 'PWRAPPDB'
        driver = '{ODBC Driver 17 for SQL Server}'

        # Establish the database connection using AAD Integrated Authentication
        conn_str = URL.create(
            'mssql+pyodbc',
            query={
                'odbc_connect':(
                    f'DRIVER={driver};'
                    f'SERVER=tcp:{server};'
                    f'DATABASE={database};'
                    'Authentication=ActiveDirectoryIntegrated;'
            )
            }
        )

        engine = create_engine(conn_str, connect_args={"autocommit": True}, fast_executemany=True, use_insertmanyvalues=False)

        return engine

        
    def insert_rows(self):
        engine = bek_sellout.setup_miengine()

        table_name = 'US_BEK_Sellout'
        schema_name = 'na_dist'

        # If the table doesn't exist, it will be created automatically
        self.data_import.to_sql(table_name, con=engine, schema=schema_name, if_exists='append', index=False)



## Individual Files

In [2]:

file_path = r'C:\Users\newatter\OneDrive - McCain Foods Limited\Distributor Sell-Out\Weekly Update Files\2024-05-10\BEK Update.xls'

bek_sellout(file_path)

In [ ]:
#Legacy Files
#C:\Users\newatter\OneDrive - McCain Foods Limited\Distributor Sell-Out\Weekly Update Files\2021-01-22\BEK Update 2019-1.xls - Done
#C:\Users\newatter\OneDrive - McCain Foods Limited\Distributor Sell-Out\Weekly Update Files\2021-01-22\BEK Update 2019-2.xls - Done
#C:\Users\newatter\OneDrive - McCain Foods Limited\Distributor Sell-Out\Weekly Update Files\2021-01-22\BEK Update 2019-3.xls - Done
#C:\Users\newatter\OneDrive - McCain Foods Limited\Distributor Sell-Out\Weekly Update Files\2021-01-22\BEK Update 2020-1.xls - Done
#C:\Users\newatter\OneDrive - McCain Foods Limited\Distributor Sell-Out\Weekly Update Files\2021-01-22\BEK Update 2020-2.xls - Having issues

## Directory - Batch Processing

In [53]:
import os

# The path to the directory containing the folders
directory_path = r'C:\Users\newatter\OneDrive - McCain Foods Limited\Distributor Sell-Out\Weekly Update Files'

# Loop through each folder and file in the directory
for root, dirs, files in os.walk(directory_path):
    for file in files:
        # Check if "SharedTable_weekly_cases" is in the filename
        if "BEK Update.xls" in file:

            # Construct the full file path
            file_path = os.path.join(root, file)
            
            print(file_path)

            bek_sellout(file_path)

C:\Users\newatter\OneDrive - McCain Foods Limited\Distributor Sell-Out\Weekly Update Files\2020-11-13\BEK Update.xls


C:\Users\newatter\AppData\Local\Temp\1\ipykernel_27072\626213147.py:148: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  return self.data[~self.data['Week Beginning Date'].isin(dates_in_db)]


C:\Users\newatter\OneDrive - McCain Foods Limited\Distributor Sell-Out\Weekly Update Files\2020-11-20\BEK Update.xls


C:\Users\newatter\AppData\Local\Temp\1\ipykernel_27072\626213147.py:148: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  return self.data[~self.data['Week Beginning Date'].isin(dates_in_db)]


C:\Users\newatter\OneDrive - McCain Foods Limited\Distributor Sell-Out\Weekly Update Files\2021-01-22\BEK Update.xls


...

C:\Users\newatter\AppData\Local\Temp\1\ipykernel_27072\626213147.py:148: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  return self.data[~self.data['Week Beginning Date'].isin(dates_in_db)]


C:\Users\newatter\OneDrive - McCain Foods Limited\Distributor Sell-Out\Weekly Update Files\2024-04-05\BEK Update.xls


C:\Users\newatter\AppData\Local\Temp\1\ipykernel_27072\626213147.py:148: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  return self.data[~self.data['Week Beginning Date'].isin(dates_in_db)]


C:\Users\newatter\OneDrive - McCain Foods Limited\Distributor Sell-Out\Weekly Update Files\2024-04-12\BEK Update.xls


C:\Users\newatter\AppData\Local\Temp\1\ipykernel_27072\626213147.py:148: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  return self.data[~self.data['Week Beginning Date'].isin(dates_in_db)]


# BEK Dictionary

In [1]:
class bek_dictionary:
    def __init__(self, file_path) -> None:
        self.filepath = file_path
        self.data = self.process_file()
        self.delete_rows()
        self.insert_rows()

        pass
    

    def process_file(self):
        df = pd.read_excel(self.filepath, sheet_name='Segment Mapping')

        columns_to_keep = [
            'Business Unit',
            'SIC Code',
            'SIC Sub',
            'COVID Segmentation - L1',	
            'COVID Segmentation - L2',
            'COVID Segmentation - (Restaurants)',
            'COVID Segmentation - (Restaurants: Sub-Segment)',
            'Restaurant Service Type',
            'Cuisine Type'
        ]

        rename_columns = {
            'COVID Segmentation - L1':'Segmentation L1',
            'COVID Segmentation - L2':'Segmentation L2',
            'COVID Segmentation - (Restaurants)':'Restaurants',
            'COVID Segmentation - (Restaurants: Sub-Segment)':'Restaurants Sub',
            'Restaurant Service Type':'Service Type'
        }
        
        df_clean = df[columns_to_keep].rename(columns=rename_columns)

        return df_clean

    def setup_connection():
        server = 'mf-enterprise-dev-sql.46ac3df1733c.database.windows.net'
        database = 'PWRAPPDB'
        driver = '{ODBC Driver 17 for SQL Server}'

        # Establish the database connection using AAD Integrated Authentication
        conn_str = (
            f'DRIVER={driver};'
            f'SERVER=tcp:{server};'
            f'DATABASE={database};'
            'Authentication=ActiveDirectoryIntegrated'
        )

        cnxn = pyodbc.connect(conn_str)
        cursor = cnxn.cursor()

        return cnxn, cursor

    def delete_rows(self):

        sql_delete = """
        DELETE FROM [na_dist].[US_BEK_Segmentation] 
        """

        cnxn, cursor = bek_dictionary.setup_connection()

        cursor.execute(sql_delete)

        # Commit the transactions
        cnxn.commit()

        # Close the connection
        cursor.close()
        cnxn.close()


    def setup_miengine():
        server = 'mf-enterprise-dev-sql.46ac3df1733c.database.windows.net'
        database = 'PWRAPPDB'
        driver = '{ODBC Driver 17 for SQL Server}'

        # Establish the database connection using AAD Integrated Authentication
        conn_str = URL.create(
            'mssql+pyodbc',
            query={
                'odbc_connect':(
                    f'DRIVER={driver};'
                    f'SERVER=tcp:{server};'
                    f'DATABASE={database};'
                    'Authentication=ActiveDirectoryIntegrated;'
            )
            }
        )

        engine = create_engine(conn_str, connect_args={"autocommit": True}, fast_executemany=True, use_insertmanyvalues=False)

        return engine

        
    def insert_rows(self):
        engine = bek_dictionary.setup_miengine()

        table_name = 'US_BEK_Segmentation'
        schema_name = 'na_dist'

        # If the table doesn't exist, it will be created automatically
        self.data.to_sql(table_name, con=engine, schema=schema_name, if_exists='append', index=False)

In [61]:
file_path = r'C:\Users\newatter\OneDrive - McCain Foods Limited\Distributor Sell-Out\Data Dictionaries\BEK - US.xlsx'

bek_dictionary(file_path)